In [188]:
import pandas as pd
import glob 
import plotly.express as px 
import plotly.io as pio


In [189]:

# find all csv files in folder "Stock Data" -
csv_files = glob.glob("/Users/jan/Techlabs_Project_Data/Stock_Data/*.csv") 
# read all csv files -> all files have the same structure
df_list = [pd.read_csv(file) for file in csv_files] 
#combine all csv files to one dataframe
stock_data = pd.concat(df_list, ignore_index=True)


In [190]:
#For POC we only use Data from Year 2008 & 2009 
stock_data['Date']=pd.to_datetime(stock_data['Date'], format= 'ISO8601', utc=True)
stock_data = stock_data[stock_data["Date"].dt.year<=2009]
#If we want to show the influence from Natural Disaster on stocks, we need the difference (in%) from Close to Open 
stock_data['percentage_difference_open_close'] = ((stock_data['Close']-stock_data['Open'])/stock_data['Open'])*100

In [191]:
#read data natural disaster
earthquakes_data = pd.read_csv('Eartquakes-1990-2023.csv')


In [192]:
#For POC we only use Data from Year 2008 & 2009. 
earthquakes_data['date']=pd.to_datetime(earthquakes_data['date'], format= 'ISO8601')
earthquakes_data = earthquakes_data[(earthquakes_data['date'].dt.year>=2008)&(earthquakes_data['date'].dt.year<=2009)]
earthquakes_data=earthquakes_data.drop(columns=['time','status','tsunami'])# Drop columns 'time', 'status', 'tsunami'


In [193]:
# prepare date columns to act as primary key 
stock_data.rename(columns={'Date':'date'},inplace=True) #rename date columns in stock_data
# Make sure that both columns are in the same datetime format
earthquakes_data["date"] = pd.to_datetime(earthquakes_data["date"]).dt.date 
stock_data["date"] = pd.to_datetime(stock_data["date"]).dt.date

In [194]:
#merge stock_data and earthquake_data to one dataframe.primary key is the date
disaster_expanded = earthquakes_data.merge(stock_data, on = 'date', how= 'outer')
disaster_expanded = disaster_expanded.sort_values(by=["date"])


In [195]:
# Convert date to datetime
disaster_expanded["date"] = pd.to_datetime(disaster_expanded["date"])  



In [196]:
#Filter the Data to all Disasters in "State" Japan & Region Japan and Japan Ticker N225 or 
#if there was a natural disaster on a day when there was no trading day

disaster_japan = disaster_expanded[
    disaster_expanded["state"].str.contains('Japan', na=False) & 
    (disaster_expanded["Ticker"].eq('^N225') | disaster_expanded["Ticker"].isna())& (disaster_expanded['magnitudo']>= 6.0)]

In [197]:
#define variabels that we want to use for the Info Points on the map 
latitude = disaster_japan["latitude"]
longitude = disaster_japan["longitude"]
magnitude = disaster_japan["magnitudo"]
depth = disaster_japan["depth"]
place = disaster_japan ['place']
date = disaster_japan ['date']

In [198]:
# Open the map in the browser
pio.renderers.default = "browser"  
#Our Mapbox Access Token 
px.set_mapbox_access_token("pk.eyJ1IjoidGVjaGxhYnMzIiwiYSI6ImNtNzBib2xyczAwZHoycnBiM2hxZ24zcngifQ.AaSHNEfc-cnR4uDdEO4gsw")

In [199]:

# Create Map with Markers for every disaster in our filtered Map for Japan & Region Japan 
fig = px.density_map(
    disaster_japan, 
    lat="latitude", 
    lon="longitude", 
    z="magnitudo",  #colour intensity after magnitude
    hover_name="place",  #first name in the markers
    zoom=4, #level of zoom 
    radius= 10, # hight of the markers on the map 
    color_continuous_scale=px.colors.sequential.Plasma_r, # Turn Color Scheme upside down
    map_style="light"  # style of the map 
)

# Karte anzeigen
fig.show()
